In [2]:
import argparse
import numpy as np
import pandas as pd
import pickle
import ast, os, sys
import neuroglancer
import matplotlib
import cv2
from skimage import img_as_ubyte
from matplotlib import pyplot as plt

In [3]:
PATH = '/home/eddyod/programming/pipeline_utility'
sys.path.append(PATH)
from utilities.contour_utilities import get_contours_from_annotations, add_structure_to_neuroglancer
neuroglancer.set_server_bind_address(bind_port='33645')

Connecting dklab@192.168.1.12:3306


In [4]:
DIR = '/net/birdstore/Active_Atlas_Data/data_root/CSHL_volumes/MD589'
ATLASPATH = os.path.join(DIR, 'MD594')
contourpath = os.path.join(DIR, 'full_brain_volume_annotated.npy')
#contourpath = os.path.join('/home/eddyod/programming/pipeline_utility/neuroglancer', 'aligned_moving_instance.npy')
contour = np.load(contourpath)

In [ ]:
contour = img_as_ubyte(contour)

In [ ]:
contour.dtype, contour.shape, np.median(contour), np.mean(contour), np.amax(contour)

In [ ]:
M = np.median(contour)
keep_mask = contour==M
vm = np.where(contour>M,0,1)
vm[keep_mask] = M
vm = vm.astype(np.uint8)
vm.dtype, vm.shape

In [ ]:
#ng_structure_volume = structure_volumes[0]
vm = np.swapaxes(contour,0,2)
#vol_m = np.swapaxes(vm, 0,1)
vm.shape, vm.dtype

In [ ]:
arr = vm
limit = 120
start = (arr.shape[2] // 2) - limit
end = start + limit * 2
nrows = end - start
ncols = 1
plt.style.use('classic')
for i in range(start, end, 4):
    plt.title('Z {}'.format(i))
    plt.imshow(arr[:,:,i], cmap="cool")
    plt.show()

In [5]:
csvfile = os.path.join(PATH, 'neuroglancer/contours','MD589_annotation_contours.csv')
MD589_annotations = pd.read_csv(csvfile)
MD589_annotations.head()

,Unnamed: 0,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices
0,100157269992733,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-IHC69-2015.07.31-02.07.15_MD589_1_0205,100157269992733,[15317.43823807 12578.50401305],VCA,sagittal,[],raw,367,R,False,9152016033619,confirmed,[[15450.64315379 13041.15143737]\n [15292.3766...
1,100472025966132,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0912201...",MD589-N51-2015.07.30-19.28.17_MD589_2_0152,100472025966132,[25707.69102116 17933.27685287],IO,sagittal,[],raw,260,R,False,9122016211223,confirmed,[[25199.46145603 17916.30221869]\n [24976.1090...
2,10063296125941,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-N55-2015.07.30-19.49.42_MD589_3_0165,10063296125941,[28722.35048812 16712.94413081],LRt,sagittal,[],raw,286,R,False,9152016024654,confirmed,[[29069.49187508 17203.67921491]\n [28909.8937...
3,101105413939417,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-N54-2015.07.30-19.44.32_MD589_1_0160,101105413939417,[28529.80042174 15814.38685363],LRt,sagittal,[],raw,276,R,False,9152016024654,confirmed,[[29219.12776439 16422.80664145]\n [29084.8059...
4,101214741356495,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0531201...",MD589-N37-2015.07.30-18.32.51_MD589_1_0109,101214741356495,[22896.75853307 6674.71411486],SC,sagittal,[],raw,202,S,False,5312018021355,intersected,[[21584.44352233 7847.78193722]\n [21497.5691...


In [8]:
MD589_annotations.groupby(['creator', 'name']).head(20)

,Unnamed: 0,class,creator,edits,filename,id,label_position,name,orientation,parent_structure,resolution,section,side,side_manually_assigned,time_created,type,vertices
0,100157269992733,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-IHC69-2015.07.31-02.07.15_MD589_1_0205,100157269992733,[15317.43823807 12578.50401305],VCA,sagittal,[],raw,367,R,False,9152016033619,confirmed,[[15450.64315379 13041.15143737]\n [15292.3766...
1,100472025966132,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0912201...",MD589-N51-2015.07.30-19.28.17_MD589_2_0152,100472025966132,[25707.69102116 17933.27685287],IO,sagittal,[],raw,260,R,False,9122016211223,confirmed,[[25199.46145603 17916.30221869]\n [24976.1090...
2,10063296125941,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-N55-2015.07.30-19.49.42_MD589_3_0165,10063296125941,[28722.35048812 16712.94413081],LRt,sagittal,[],raw,286,R,False,9152016024654,confirmed,[[29069.49187508 17203.67921491]\n [28909.8937...
3,101105413939417,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-N54-2015.07.30-19.44.32_MD589_1_0160,101105413939417,[28529.80042174 15814.38685363],LRt,sagittal,[],raw,276,R,False,9152016024654,confirmed,[[29219.12776439 16422.80664145]\n [29084.8059...
4,101214741356495,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0531201...",MD589-N37-2015.07.30-18.32.51_MD589_1_0109,101214741356495,[22896.75853307 6674.71411486],SC,sagittal,[],raw,202,S,False,5312018021355,intersected,[[21584.44352233 7847.78193722]\n [21497.5691...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2256,97596949301437,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0728201...",MD589-N29-2015.07.30-17.58.29_MD589_3_0087,97596949301437,[25835.74487726 8494.50229677],Cb,sagittal,[],raw,170,L,False,7282017180721,confirmed,[[22424.46783584 9542.48750523]\n [22476.3301...
2257,97618404545778,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-IHC62-2015.07.31-01.39.26_MD589_2_0185,97618404545778,[16758.58730184 13725.14566634],Pr5VL,sagittal,[],raw,327,R,False,9152016032437,confirmed,[[17130.38926046 14768.12089952]\n [16988.3572...
2258,97664229774079,contour,lauren,"[{'username': 'lauren', 'timestamp': '05302018...",MD589-N52-2015.07.30-19.33.44_MD589_1_0154,97664229774079,[20817.61343609 12758.07943283],SNR,sagittal,[],raw,264,R,False,5302018164915,confirmed,[[20541.00254679 13008.50911521]\n [20472.5689...
2259,97857026901462,contour,yuncong,"[{'username': 'yuncong', 'timestamp': '0915201...",MD589-IHC58-2015.07.31-01.22.39_MD589_2_0173,97857026901462,[18993.2564127 15422.18959153],LSO,sagittal,[],raw,303,R,False,9152016024520,confirmed,[[19020.39355658 15711.24642154]\n [18886.0601...


In [ ]:
hand_annotations.head(4)

In [ ]:
animal = 'atlasV7'
target_structure = 'Tz'
str_contours_annotation, first_sec, last_sec = get_contours_from_annotations(animal, target_structure, hand_annotations, densify=0)

In [ ]:
first_sec, last_sec

In [ ]:
dimensions = neuroglancer.CoordinateSpace(
    names=['x', 'y', 'z'],
    units='nm',
    scales=[10, 10, 10])

viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.dimensions = dimensions
    s.layers['Altas'] = neuroglancer.ImageLayer(
        source=neuroglancer.LocalVolume(
            data=contour,
            dimensions=neuroglancer.CoordinateSpace(
                names=['x', 'y', 'z'],
                units=['nm','nm','nm'],
                scales=[5, 5, 5]),
            voxel_offset=(0, 0, 0),
        ),
        shader="""
#uicontrol float min slider(min=0, max=1, default=0)
#uicontrol float max slider(min=0, max=1, default=1)
#uicontrol float invert slider(min=0, max=1, default=0, step=1)
#uicontrol float brightness slider(min=-1, max=1)
#uicontrol float contrast slider(min=-3, max=3, step=0.01)
void main() {
  float pix_val = toNormalized(getDataValue());
  if(pix_val < min){
  	pix_val = 0.0;
  }
  if(pix_val > max){
    pix_val = 1.0;
  }

  if(invert==1.0){
  	  emitGrayscale((1.0 -(pix_val - brightness)) *
       exp(contrast));
  }
  else{
    emitGrayscale((pix_val + brightness) *
                  exp(contrast));
  }

}

""")
print(viewer)

In [ ]:
csvfile = os.path.join(PATH, 'neuroglancer/contours', 'hand_annotations.csv')
hand_annotations = pd.read_csv(csvfile)
hand_annotations['vertices'] = hand_annotations['vertices'].apply(lambda x: ast.literal_eval(x))
animal = 'MD589'
color_codes = {"3N": 3, "4N": 4, "5N": 5, "6N": 6, "Tz": 24, "VCA_L": 25, "VCA_R": 25}
viewer = neuroglancer.Viewer()
# Sets 'Image' layer to be MD585 prep2 images from S3
with viewer.txn() as s:
    s.layers[animal] = neuroglancer.ImageLayer(source='precomputed://https://mousebrainatlas-datajoint-jp2k.s3.amazonaws.com/precomputed/'+animal+'_fullres')
    s.layout = 'xy' # '3d'/'4panel'/'xy'
print(viewer)

In [ ]:
neuroglancer.stop()

In [ ]:
structure_volumes = []
offsets = []
structures = ['3N', '4N', '5N', '6N', 'Tz']
for target_structure in structures:
    color = color_codes[target_structure]
    str_contours_annotation, first_sec, last_sec = get_contours_from_annotations(animal, target_structure, hand_annotations, densify=0)
    ng_structure_volume, xyz_offsets = add_structure_to_neuroglancer(viewer, str_contours_annotation, target_structure, animal, first_sec, last_sec, \
                                                        color_radius=2, xy_ng_resolution_um=5, threshold=1, color=color, \
                                                        solid_volume=False, no_offset_big_volume=True, save_results=False, \
                                                        return_with_offsets=True, add_to_ng=True, human_annotation=True )
    structure_volumes.append(ng_structure_volume)
    offsets.append(xyz_offsets)

In [ ]:
print(viewer)

In [ ]:
structures = ['3N', '4N', '5N', '6N', 'Tz']
for structure, volume, offset in zip(structures, structure_volumes,offsets):
    vm = np.swapaxes(volume,0,2)
    print(structure, vm.shape, offset)